In [12]:
import pandas as pd
import requests
import re

# Function to remove isolated letters followed by a period and space
def clean_name(name):
    return re.sub(r'\b[A-Z]\.\s+', '', name)

# Function to get Wikipedia link for a given name, including country, state, and party
def get_wikipedia_link(row):
    name = row['Name']
    country = row['Country']
    state = row['State']
    party = row['Party']
    
    search_queries = [
        f"{name} {country} politician",
        f"{name} {state} politician",
        f"{name} {party} politician"
    ]
    
    for query in search_queries:
        search_url = f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={query}&format=json"
        response = requests.get(search_url).json()
        
        try:
            # Get the first search result page ID
            page_id = response['query']['search'][0]['pageid']
            page_url = f"https://en.wikipedia.org/?curid={page_id}"
            
            # Get the content of the page
            page_content_url = f"https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exintro&explaintext&format=json&pageids={page_id}"
            page_content = requests.get(page_content_url).json()
            extract = page_content['query']['pages'][str(page_id)]['extract']
            
            # Check if the page mentions the person as a politician
            if 'politician' in extract.lower():
                return page_url
        except (IndexError, KeyError):
            continue
    
    return None

# Load and process the Control dataset
control_file_path = 'C:/Users/Guill/OneDrive/Documents/Control.csv'  # Replace with the path to your actual Control dataset
control_df = pd.read_csv(control_file_path, encoding='ISO-8859-1')

# Remove leading and trailing spaces from the 'Name' column
control_df['Name'] = control_df['Name'].str.strip()

# Filter out rows that contain 'Ã' in any column
control_df = control_df[~control_df.apply(lambda row: row.astype(str).str.contains('Ã').any(), axis=1)]

# Clean names in the Control dataset
control_df['Name'] = control_df['Name'].apply(clean_name)

# Apply the Wikipedia link function to the Control dataset
control_df['Wikipedia_Link'] = control_df.apply(get_wikipedia_link, axis=1)

# Save the updated Control DataFrame to a new CSV file
control_output_file_path = 'C:/Users/Guill/OneDrive/Documents/Control_with_links.csv'
control_df.to_csv(control_output_file_path, index=False)

# Load and process the Treatment dataset
treatment_file_path = 'C:/Users/Guill/OneDrive/Documents/Treatment.csv'  # Replace with the path to your actual Treatment dataset
treatment_df = pd.read_csv(treatment_file_path, encoding='ISO-8859-1')

# Remove leading and trailing spaces from the 'Name' column
treatment_df['Name'] = treatment_df['Name'].str.strip()

# Filter out rows that contain 'Ã' in any column
treatment_df = treatment_df[~treatment_df.apply(lambda row: row.astype(str).str.contains('Ã').any(), axis=1)]

# Clean names in the Treatment dataset
treatment_df['Name'] = treatment_df['Name'].apply(clean_name)

# Apply the Wikipedia link function to the Treatment dataset
treatment_df['Wikipedia_Link'] = treatment_df.apply(get_wikipedia_link, axis=1)

# Save the updated Treatment DataFrame to a new CSV file
treatment_output_file_path = 'C:/Users/Guill/OneDrive/Documents/Treatment_with_links.csv'
treatment_df.to_csv(treatment_output_file_path, index=False)

# Display the updated DataFrames
print(control_df)
print(treatment_df)


                 Name         Party Country       State              Date in  \
0     Donald McEachin    Democratic     USA    Virginia  2017-01-01 00:00:00   
1       Drew Ferguson    Republican     USA     Georgia  2017-01-01 00:00:00   
2          Aaron Bean    Republican     USA     Florida  2023-01-01 00:00:00   
3        Aaron Schock    Republican     USA    Illinois  2009-01-01 00:00:00   
4     Abby Finkenauer    Democratic     USA        Iowa  2019-01-01 00:00:00   
...               ...           ...     ...         ...                  ...   
3265        Zach Wamp    Republican     USA   Tennessee  1995-01-01 00:00:00   
3266     Zachary Nunn    Republican     USA        Iowa  2023-01-01 00:00:00   
3267    Zachary Space    Democratic     USA        Ohio  2007-01-01 00:00:00   
3268   Ziad Aboultaif  Conservative  Canada     Alberta  2015-12-03 00:00:00   
3269      Zoe Lofgren    Democratic     USA  California  1995-01-01 00:00:00   

        Date Out                       

In [13]:
print(treatment_df)


     Country                  Name          State Date of party switch  \
0         UK        Zahida Manzoor        England               16-Sep   
1         UK         William Wragg        England            09-Apr-24   
2         UK         William Legge        England               18-Sep   
3        USA        William Daniel     Louisiana                  2005   
4        USA  Wanda Vázquez Garced   Puerto Rico                  2019   
..       ...                   ...            ...                  ...   
521  Canada             Anne Cools       Ontario            08-Jun-04    
522  Canada            Andre Riedl        Quebec                  2008   
523  Canada       André Bellavance        Quebec                  2014   
524  Canada            Amir Khadir        Quebec                  2008   
525  Canada          Amanda Simard       Ontario                  2020   

                 Party          New Party  \
0    Liberal Democrats       Conservative   
1         Conservativ

In [78]:

# Convert 'Date in' and 'Date Out' to datetime in both DataFrames
control_df['Date in'] = pd.to_datetime(control_df['Date in'], errors='coerce')
control_df['Date Out'] = pd.to_datetime(control_df['Date Out'], errors='coerce')

# Function to merge duplicate records
def merge_duplicates(df):
    merged_df = df.groupby(['Name', 'Party', 'Country', 'State', 'Wikipedia_Link']).agg({
        'Date in': 'min',
        'Date Out': 'max'
    }).reset_index()
    return merged_df

# Remove duplicates in both DataFrames
control_df = merge_duplicates(control_df)

# Display the head of the DataFrames to verify
print("control_df after removing duplicates:")
print(control_df.head())




control_df after removing duplicates:
                 Name       Party Country     State  \
0  A. Donald McEachin  Democratic     USA  Virginia   
1    A. Drew Ferguson  Republican     USA   Georgia   
2          Aaron Bean  Republican     USA   Florida   
3        Aaron Schock  Republican     USA  Illinois   
4     Abby Finkenauer  Democratic     USA      Iowa   

                             Wikipedia_Link    Date in   Date Out  
0   https://en.wikipedia.org/?curid=3353264 2017-01-01 2022-01-01  
1  https://en.wikipedia.org/?curid=51189954 2017-01-01        NaT  
2  https://en.wikipedia.org/?curid=30869022 2023-01-01        NaT  
3   https://en.wikipedia.org/?curid=7150289 2009-01-01 2015-01-01  
4  https://en.wikipedia.org/?curid=50000800 2019-01-01 2021-01-01  


In [81]:
# Convert 'Date in' and 'Date Out' to datetime
control_df['Date in'] = pd.to_datetime(control_df['Date in'], errors='coerce')
control_df['Date Out'] = pd.to_datetime(control_df['Date Out'], errors='coerce')
Treatment_df['Date in'] = pd.to_datetime(Treatment_df['Date in'], errors='coerce')
Treatment_df['Date Out'] = pd.to_datetime(Treatment_df['Date Out'], errors='coerce')

# Identify matching Wikipedia links
matching_links = control_df['Wikipedia_Link'].isin(Treatment_df['Wikipedia_Link'])

# Remove matching entries from control_df
non_matching_control_df = control_df[~matching_links]

# For non-matching control entries, set old party and new party to the same value
non_matching_control_df['Old party'] = non_matching_control_df['Party']
non_matching_control_df['New party'] = non_matching_control_df['Party']
non_matching_control_df['Date of party switch'] = pd.NaT  # No party switch for control

# Select relevant columns from non-matching control entries
non_matching_control_df = non_matching_control_df[['Country', 'Name', 'State', 'Date in', 'Date Out', 'Wikipedia_Link', 'Old party', 'New party', 'Date of party switch']]

# Ensure Treatment_df has the required columns
Treatment_df = Treatment_df[['Country', 'Name', 'State', 'Date in', 'Date Out', 'Wikipedia_Link', 'Old party', 'New party', 'Date of party switch']]

# Merge control_df into Treatment_df
merged_df = pd.concat([Treatment_df, non_matching_control_df], ignore_index=True)

# Display the head of the merged DataFrame
merged_df



C:\Users\Guill\AppData\Local\Temp\ipykernel_18832\3028066363.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_matching_control_df['Old party'] = non_matching_control_df['Party']
C:\Users\Guill\AppData\Local\Temp\ipykernel_18832\3028066363.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_matching_control_df['New party'] = non_matching_control_df['Party']
C:\Users\Guill\AppData\Local\Temp\ipykernel_18832\3028066363.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,Country,Name,State,Date in,Date Out,Wikipedia_Link,Old party,New party,Date of party switch
0,Canada,A. Donald McEachin,Ontario,NaT,NaT,None,Conservative,Liberal,2005
1,Canada,A. Drew Ferguson,Quebec,NaT,NaT,None,Liberal,Conservative,2005
2,Canada,Aaron Bean,Quebec,NaT,NaT,None,Marijuana Party,Liberal Party,2005
3,Canada,Aaron Schock,Territories,NaT,NaT,https://en.wikipedia.org/?curid=25616173,Yukon Party,NDP,2006
4,Canada,Abby Finkenauer,Ontario,NaT,NaT,None,NDP,Liberal,2006
...,...,...,...,...,...,...,...,...,...
3370,USA,Zach Wamp,Tennessee,1995-01-01,2011-01-01,https://en.wikipedia.org/?curid=960873,Republican,Republican,NaN
3371,USA,Zachary Nunn,Iowa,2023-01-01,NaT,https://en.wikipedia.org/?curid=67576919,Republican,Republican,NaN
3372,USA,Zachary T. Space,Ohio,2007-01-01,2011-01-01,https://en.wikipedia.org/?curid=4058815,Democratic,Democratic,NaN
3373,Canada,Ziad Aboultaif,Alberta,2015-12-03,2021-08-15,https://en.wikipedia.org/?curid=48297891,Conservative,Conservative,NaN


In [86]:
# Remove observations without both Name and Wikipedia_Link
merged_df = merged_df.dropna(subset=['Name', 'Wikipedia_Link'], how='all')
merged_df.to_csv("C:/Users/Guill/OneDrive/Documents/merged.csv") 